In [1]:
#全てのテキストを巡回して単語データベースを作成する
import os, glob
import MeCab
import numpy as np
import pickle

#保存ファイル名
savefile = "./ok-spam.pickle"
#MeCabの準備
tagger = MeCab.Tagger()
#変数の準備
word_dic = {"__id": 0} #単語辞書
files = [] #読み込んだ単語データを追加する

#指定したディレクトリー内のファイル一覧を読む
def read_files(dir, label):
    files = glob.glob(dir + '/*.txt')
    for f in files:
        read_file(f, label)

#ファイルを読む
def read_file(filename, label):
    words = []
    #ファイルの内容を読む
    with open(filename, "rt", encoding="utf-8") as f:
        text = f.read()
    files.append({
        "label" : label,
        "words" : text_to_ids(text)
    })
    
#テキストを単語IDリストに変換
def text_to_ids(text):
    #形態素解析
    word_s = tagger.parse(text)
    words = []
    #単語を辞書に登録
    for line in word_s.split("\n"):
        if line == 'EOS' or line == '':
            continue
        word = line.split("\t")[0]
        params = line.split("\t")[4].split("-")
        hinsi = params[0]
        hinsi2 = params[1] if len(params) > 1 else ''
        org = line.split("\t")[3]
        #助詞・助動詞・記号・数字は捨てる
        if not (hinsi in ['名詞', '動詞', '形容詞']) :
            continue
        if hinsi == '名詞' and hinsi2 == '数詞':
            continue
        #単語をIDに変換
        id = word_to_id(org)
        words.append(id)
    return words

#単語をIDに変換
def word_to_id(word):
    #単語が辞書に登録されているか
    if not (word in word_dic):
        #登録されていないので新たにIDを割り振る
        id = word_dic["__id"]
        word_dic["__id"] += 1
        word_dic[word] = id
    else:
        #既存の単語IDを返す
        id = word_dic[word]
    return id

#単語の出現頻度のデータを作る
def make_freq_data_allfiles():
    y = []
    x = []
    for f in files:
        y.append(f['label'])
        x.append(make_freq_data(f['words']))
    return y, x

def make_freq_data(words):
    #単語の出現回数を調べる
    cnt = 0
    dat = np.zeros(word_dic["__id"], 'float')
    for w in words:
        dat[w] += 1
        cnt += 1
    #回数を出現頻度に直す
    dat = dat / cnt
    return dat

#ファイルの一覧から学習用のデータベースを作る
if __name__ == "__main__":
    read_files("ok", 0)
    read_files("spam", 1)
    y, x = make_freq_data_allfiles()
    #ファイルをデータに保存
    pickle.dump([y, x, word_dic], open(savefile, 'wb'))
    print("単語頻出データ作成完了")

単語頻出データ作成完了
